In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [2]:
fake_df = pd.read_csv("Fake.csv")
true_df = pd.read_csv("True.csv")

print("Fake news shape:", fake_df.shape)
print("True news shape:", true_df.shape)

fake_df.head()

Fake news shape: (23481, 4)
True news shape: (21417, 4)


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
fake_df["label"] = 0
true_df["label"] = 1

fake_df.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [6]:
true_df.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [7]:
#merging both dataframes
df = pd.concat([fake_df, true_df], axis=0)
df = df.sample(frac=1).reset_index(drop=True)  

print(df.shape)
df.head()

(44898, 5)


,title,text,subject,date,label
0,Trump says he is close to picking new FBI dire...,WASHINGTON (Reuters) - President Donald Trump ...,politicsNews,"May 18, 2017",1
1,WOW! NEW VIDEO BLOWS UP CORRUPTION Between Oba...,"These videos, by James O Keefe and friends at ...",politics,"Oct 26, 2016",0
2,Fighting travel ban is gamble for Democratic o...,WASHINGTON/SAN FRANCISCO (Reuters) - Three of ...,politicsNews,"February 8, 2017",1
3,"Gorka, a Trump adviser and Bannon ally, is out...",WASHINGTON (Reuters) - White House adviser Seb...,politicsNews,"August 26, 2017",1
4,Clinton expands on plan to tax wealthy as Sand...,"AMES, Iowa (Reuters) - U.S. Democratic preside...",politicsNews,"January 13, 2016",1


In [8]:
df = df[["text", "label"]]
df.head()

,text,label
0,WASHINGTON (Reuters) - President Donald Trump ...,1
1,"These videos, by James O Keefe and friends at ...",0
2,WASHINGTON/SAN FRANCISCO (Reuters) - Three of ...,1
3,WASHINGTON (Reuters) - White House adviser Seb...,1
4,"AMES, Iowa (Reuters) - U.S. Democratic preside...",1


In [9]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\patil\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [10]:
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))

def clean_text(text):
    text = text.lower()                     
    text = re.sub(r'\[.*?\]', '', text)      
    text = re.sub(r'http\S+|www\S+', '', text)  
    text = re.sub(r'<.*?>+', '', text)       
    text = re.sub(r'[^a-z\s]', '', text)     
    text = re.sub(r'\n', ' ', text)          
    
    words = text.split()
    words = [word for word in words if word not in stop_words]
    
    return " ".join(words)


In [11]:
df["clean_text"] = df["text"].apply(clean_text)
df.head()

,text,label,clean_text
0,WASHINGTON (Reuters) - President Donald Trump ...,1,washington reuters president donald trump said...
1,"These videos, by James O Keefe and friends at ...",0,videos james keefe friends project veritas bus...
2,WASHINGTON/SAN FRANCISCO (Reuters) - Three of ...,1,washingtonsan francisco reuters three democrat...
3,WASHINGTON (Reuters) - White House adviser Seb...,1,washington reuters white house adviser sebasti...
4,"AMES, Iowa (Reuters) - U.S. Democratic preside...",1,ames iowa reuters us democratic presidential c...


In [12]:
X = df["clean_text"]
y = df["label"]

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X_vectorized = vectorizer.fit_transform(X)
print("Shape of X after vectorization:", X_vectorized.shape)


Shape of X after vectorization: (44898, 5000)


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_vectorized, y, test_size=0.2, random_state=42
)

print("Training samples:", X_train.shape)
print("Testing samples:", X_test.shape)


Training samples: (35918, 5000)
Testing samples: (8980, 5000)


In [15]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

print("Model training completed")


Model training completed


In [16]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9900890868596882
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4652
           1       0.99      0.99      0.99      4328

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [17]:
def predict_news(news_text):
    cleaned = clean_text(news_text)
    vector = vectorizer.transform([cleaned])
    prediction = model.predict(vector)
    
    if prediction[0] == 0:
        return "❌ Fake News"
    else:
        return "✅ Real News"


In [18]:
test_news = """
Breaking: Government announces free laptops to all students from next month.
"""

print(predict_news(test_news))


❌ Fake News


In [21]:
true_news = """
The Reserve Bank of India kept the repo rate unchanged at 6.5% in its latest monetary policy meeting, 
citing stable inflation and steady economic growth.
"""

print(predict_news(true_news))


❌ Fake News


In [20]:
true_news2 = """
NASA announced that its new space telescope has successfully captured the most detailed images yet 
of a distant galaxy, helping scientists study the early universe.
"""

print(predict_news(true_news2))


❌ Fake News
